In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from openai import OpenAI
import pandas as pd

client = OpenAI()

In [12]:
MODEL_NAME = "gpt-4o-mini"

In [13]:
## Use GPT-4 to generate synthetic data
# Define the system prompt and user input (these should be filled as per the specific use case)
system_prompt = """You are a helpful assistant that can ask questions about a database table and write SQL queries to answer the question.
    A user will pass in a table schema and your job is to return a question answer pairing. The question should relevant to the schema of the table,
    and you can speculate on its contents. You will then have to generate a SQL query to answer the question. Below are some examples of what this should look like.

    Example 1
    ```````````
    User input: Table museum, columns = [*,Museum_ID,Name,Num_of_Staff,Open_Year]\nTable visit, columns = [*,Museum_ID,visitor_ID,Num_of_Ticket,Total_spent]\nTable visitor, columns = [*,ID,Name,Level_of_membership,Age]\nForeign_keys = [visit.visitor_ID = visitor.ID,visit.Museum_ID = museum.Museum_ID]\n
    Assistant Response:
    Q: How many visitors have visited the museum with the most staff?
    A: SELECT count ( * )  FROM VISIT AS T1 JOIN MUSEUM AS T2 ON T1.Museum_ID   =   T2.Museum_ID WHERE T2.Num_of_Staff   =   ( SELECT max ( Num_of_Staff )  FROM MUSEUM ) 
    ```````````

    Example 2
    ```````````
    User input: Table museum, columns = [*,Museum_ID,Name,Num_of_Staff,Open_Year]\nTable visit, columns = [*,Museum_ID,visitor_ID,Num_of_Ticket,Total_spent]\nTable visitor, columns = [*,ID,Name,Level_of_membership,Age]\nForeign_keys = [visit.visitor_ID = visitor.ID,visit.Museum_ID = museum.Museum_ID]\n
    Assistant Response:
    Q: What are the names who have a membership level higher than 4?
    A: SELECT Name   FROM VISITOR AS T1 WHERE T1.Level_of_membership   >   4 
    ```````````

    Example 3
    ```````````
    User input: Table museum, columns = [*,Museum_ID,Name,Num_of_Staff,Open_Year]\nTable visit, columns = [*,Museum_ID,visitor_ID,Num_of_Ticket,Total_spent]\nTable visitor, columns = [*,ID,Name,Level_of_membership,Age]\nForeign_keys = [visit.visitor_ID = visitor.ID,visit.Museum_ID = museum.Museum_ID]\n
    Assistant Response:
    Q: How many tickets of customer id 5?
    A: SELECT count ( * )  FROM VISIT AS T1 JOIN VISITOR AS T2 ON T1.visitor_ID   =   T2.ID WHERE T2.ID   =   5 
    ```````````
    """

user_input = "Table car_makers, columns = [*,Id,Maker,FullName,Country]\nTable car_names, columns = [*,MakeId,Model,Make]\nTable cars_data, columns = [*,Id,MPG,Cylinders,Edispl,Horsepower,Weight,Accelerate,Year]\nTable continents, columns = [*,ContId,Continent]\nTable countries, columns = [*,CountryId,CountryName,Continent]\nTable model_list, columns = [*,ModelId,Maker,Model]\nForeign_keys = [countries.Continent = continents.ContId,car_makers.Country = countries.CountryId,model_list.Maker = car_makers.Id,car_names.Model = model_list.Model,cars_data.Id = car_names.MakeId]"

messages = [{
        "role": "system",
        "content": system_prompt
    },
    {
        "role": "user",
        "content": user_input
    }
]

completion = client.chat.completions.create(
    model=MODEL_NAME,
    messages=messages,
    temperature=0.7,
    n=5
)

for choice in completion.choices:
    print(choice.message.content + "\n")


Q: Which car maker has the highest average horsepower among their cars?  
A: SELECT Maker, AVG(Horsepower) AS AverageHorsepower FROM car_makers AS CM JOIN model_list AS ML ON CM.Id = ML.Maker JOIN car_names AS CN ON ML.Model = CN.Make JOIN cars_data AS CD ON CN.MakeId = CD.Id GROUP BY Maker ORDER BY AverageHorsepower DESC LIMIT 1;

Q: Which car maker has the highest average horsepower across all their models?
A: SELECT Maker, AVG(Horsepower) as Average_Horsepower 
   FROM car_makers AS CM 
   JOIN model_list AS ML ON CM.Id = ML.Maker 
   JOIN car_names AS CN ON ML.Model = CN.Make 
   JOIN cars_data AS CD ON CN.MakeId = CD.Id 
   GROUP BY Maker 
   ORDER BY Average_Horsepower DESC 
   LIMIT 1

Q: Which car maker has the highest horsepower among their cars?

A: SELECT Maker, MAX(Horsepower) AS Highest_Horsepower 
   FROM car_makers AS CM 
   JOIN model_list AS ML ON CM.Id = ML.Maker 
   JOIN car_names AS CN ON ML.Model = CN.Make 
   JOIN cars_data AS CD ON CN.Model = CD.Id 
   GROUP BY M

In [14]:
eval_data = []
input_prompt = "TASK: Answer the following question with syntactically correct SQLite SQL. The SQL should be correct and be in context of the previous question-answer pairs.\nTable car_makers, columns = [*,Id,Maker,FullName,Country]\nTable car_names, columns = [*,MakeId,Model,Make]\nTable cars_data, columns = [*,Id,MPG,Cylinders,Edispl,Horsepower,Weight,Accelerate,Year]\nTable continents, columns = [*,ContId,Continent]\nTable countries, columns = [*,CountryId,CountryName,Continent]\nTable model_list, columns = [*,ModelId,Maker,Model]\nForeign_keys = [countries.Continent = continents.ContId,car_makers.Country = countries.CountryId,model_list.Maker = car_makers.Id,car_names.Model = model_list.Model,cars_data.Id = car_names.MakeId]"

for choice in completion.choices:
    question = choice.message.content.split("Q: ")[1].split("\n")[0]  # Extracting the question
    answer = choice.message.content.split("\nA: ")[1].split("\n")[0]  # Extracting the answer
    eval_data.append({
        "input": [
            {"role": "system", "content": input_prompt},
            {"role": "user", "content": question},
        ],
        "ideal": answer
    })

for item in eval_data:
    print(item)

{'input': [{'role': 'system', 'content': 'TASK: Answer the following question with syntactically correct SQLite SQL. The SQL should be correct and be in context of the previous question-answer pairs.\nTable car_makers, columns = [*,Id,Maker,FullName,Country]\nTable car_names, columns = [*,MakeId,Model,Make]\nTable cars_data, columns = [*,Id,MPG,Cylinders,Edispl,Horsepower,Weight,Accelerate,Year]\nTable continents, columns = [*,ContId,Continent]\nTable countries, columns = [*,CountryId,CountryName,Continent]\nTable model_list, columns = [*,ModelId,Maker,Model]\nForeign_keys = [countries.Continent = continents.ContId,car_makers.Country = countries.CountryId,model_list.Maker = car_makers.Id,car_names.Model = model_list.Model,cars_data.Id = car_names.MakeId]'}, {'role': 'user', 'content': 'Which car maker has the highest average horsepower among their cars?  '}], 'ideal': 'SELECT Maker, AVG(Horsepower) AS AverageHorsepower FROM car_makers AS CM JOIN model_list AS ML ON CM.Id = ML.Maker JOI

In [15]:
eval_data

[{'input': [{'role': 'system',
    'content': 'TASK: Answer the following question with syntactically correct SQLite SQL. The SQL should be correct and be in context of the previous question-answer pairs.\nTable car_makers, columns = [*,Id,Maker,FullName,Country]\nTable car_names, columns = [*,MakeId,Model,Make]\nTable cars_data, columns = [*,Id,MPG,Cylinders,Edispl,Horsepower,Weight,Accelerate,Year]\nTable continents, columns = [*,ContId,Continent]\nTable countries, columns = [*,CountryId,CountryName,Continent]\nTable model_list, columns = [*,ModelId,Maker,Model]\nForeign_keys = [countries.Continent = continents.ContId,car_makers.Country = countries.CountryId,model_list.Maker = car_makers.Id,car_names.Model = model_list.Model,cars_data.Id = car_names.MakeId]'},
   {'role': 'user',
    'content': 'Which car maker has the highest average horsepower among their cars?  '}],
  'ideal': 'SELECT Maker, AVG(Horsepower) AS AverageHorsepower FROM car_makers AS CM JOIN model_list AS ML ON CM.Id 

In [17]:
import json
with open('data.jsonl', 'w') as f:
    for item in eval_data:
        f.write(json.dumps(item) + '\n')

In [18]:
import pandas as pd


In [19]:
file_path = "/Users/varadkhonde/Work/evals/evals/registry/data/chunk_metrics/Gold Set Test_ Iteration - 1.xlsx"

In [22]:
df = pd.read_excel(file_path)
df.head()

,Sl No.,Question,Answer,Book Name,Chunks (old),Book Position,Book Name.1,Chunks (new),Book Position.1,Accuracy Score,Konproz Comments
0,1,What are the GST implications of actionable cl...,The taxability of ‘actionable claims’ under GS...,GST - Law & Procedure - Volume 1 Page 101,"Section 2(1) of the CGST Act, 2017\n\n“Actiona...",Page 101 2nd definition,GST-Vol. 1,Practice and Proced ure Division 4 \n(ii) Furt...,Page 939,NaN,NaN
1,1,What are the GST implications of actionable cl...,NaN,GST Smart Guide Page 920,2. Scope of Actionable Claims under GST\nGST l...,2nd Para on page 920,GST-Vol. 1,Concept of Supply including Intra -State & Int...,Page 940,NaN,NaN
2,1,What are the GST implications of actionable cl...,NaN,GST Ready Reckoner Page 342,For 01.07.2017 to 30.09.2023\n\nActionable cla...,Para 2.1.11.8,GST-RR-10th edn.,Concept of supply including Intra -State & Int...,Page 414,NaN,NaN
3,1,What are the GST implications of actionable cl...,NaN,NaN,NaN,NaN,GST-Vol. 1,"Ltd., reported as [2019] 102 taxmann.com 286, ...",941,NaN,NaN
4,1,What are the GST implications of actionable cl...,NaN,NaN,NaN,NaN,Chapter10.,Notified Goods under RCM 93 \nCommentary \n G...,122,NaN,NaN


In [58]:
df.columns

Index(['Sl No.', 'Question', 'Answer', 'Book Name', 'Chunks (old)',
       'Book Position', 'Book Name.1', 'Chunks (new)', 'Book Position.1',
       'Accuracy Score', 'Konproz Comments'],
      dtype='object')

In [60]:
(df.groupby('Sl No.')).head()

,Sl No.,Question,Answer,Book Name,Chunks (old),Book Position,Book Name.1,Chunks (new),Book Position.1,Accuracy Score,Konproz Comments
0,1,What are the GST implications of actionable cl...,The taxability of ‘actionable claims’ under GS...,GST - Law & Procedure - Volume 1 Page 101,"Section 2(1) of the CGST Act, 2017\n\n“Actiona...",Page 101 2nd definition,GST-Vol. 1,Practice and Proced ure Division 4 \n(ii) Furt...,Page 939,NaN,NaN
1,1,What are the GST implications of actionable cl...,NaN,GST Smart Guide Page 920,2. Scope of Actionable Claims under GST\nGST l...,2nd Para on page 920,GST-Vol. 1,Concept of Supply including Intra -State & Int...,Page 940,NaN,NaN
2,1,What are the GST implications of actionable cl...,NaN,GST Ready Reckoner Page 342,For 01.07.2017 to 30.09.2023\n\nActionable cla...,Para 2.1.11.8,GST-RR-10th edn.,Concept of supply including Intra -State & Int...,Page 414,NaN,NaN
3,1,What are the GST implications of actionable cl...,NaN,NaN,NaN,NaN,GST-Vol. 1,"Ltd., reported as [2019] 102 taxmann.com 286, ...",941,NaN,NaN
4,1,What are the GST implications of actionable cl...,NaN,NaN,NaN,NaN,Chapter10.,Notified Goods under RCM 93 \nCommentary \n G...,122,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
200,41,What is TDS under GST? What is the monthly ret...,"As per Section 51 of the CGST Act, tax at sour...",Smart Guide,2. Provision of TDS Deduction\nAs per Section ...,Page 405,GST-Vol. 2,4.884 Practice and Procedure Division 4 \n(vii...,327,NaN,NaN
201,41,What is TDS under GST? What is the monthly ret...,NaN,Smart Guide,13. Monthly Return under TDS\nThe deductor is ...,Page 408,GST-Vol. 2,4.888 Practice and Procedure Division 4 \nNatu...,331,NaN,NaN
202,41,What is TDS under GST? What is the monthly ret...,NaN,NaN,NaN,NaN,"GST-RR-10th edn., July 2024",Chapter 1 0 Time of Supply and Payment of Tax ...,618,NaN,NaN
203,41,What is TDS under GST? What is the monthly ret...,NaN,NaN,NaN,NaN,GST-Vol. 1.,4.40 Practice and Procedure Division 4 \n1.22 ...,879,NaN,NaN


In [64]:
input_prompt = "As an expert legal consultant, provide a detailed response to the question below, considering the given context: {context}"

In [62]:
inputs = [{"role": "system", "content": input_prompt}]

In [152]:
old_rag_data = []
new_rag_data = []

In [153]:
df['Answer'] = df['Answer'].ffill()

In [154]:
df.head(10)

,Sl No.,Question,Answer,Book Name,Chunks (old),Book Position,Book Name.1,Chunks (new),Book Position.1,Accuracy Score,Konproz Comments
0,1,What are the GST implications of actionable cl...,The taxability of ‘actionable claims’ under GS...,GST - Law & Procedure - Volume 1 Page 101,"Section 2(1) of the CGST Act, 2017\n\n“Actiona...",Page 101 2nd definition,GST-Vol. 1,Practice and Proced ure Division 4 \n(ii) Furt...,Page 939,NaN,NaN
1,1,What are the GST implications of actionable cl...,The taxability of ‘actionable claims’ under GS...,GST Smart Guide Page 920,2. Scope of Actionable Claims under GST\nGST l...,2nd Para on page 920,GST-Vol. 1,Concept of Supply including Intra -State & Int...,Page 940,NaN,NaN
2,1,What are the GST implications of actionable cl...,The taxability of ‘actionable claims’ under GS...,GST Ready Reckoner Page 342,For 01.07.2017 to 30.09.2023\n\nActionable cla...,Para 2.1.11.8,GST-RR-10th edn.,Concept of supply including Intra -State & Int...,Page 414,NaN,NaN
3,1,What are the GST implications of actionable cl...,The taxability of ‘actionable claims’ under GS...,NaN,NaN,NaN,GST-Vol. 1,"Ltd., reported as [2019] 102 taxmann.com 286, ...",941,NaN,NaN
4,1,What are the GST implications of actionable cl...,The taxability of ‘actionable claims’ under GS...,NaN,NaN,NaN,Chapter10.,Notified Goods under RCM 93 \nCommentary \n G...,122,NaN,NaN
5,2,Explain the concept of intermediary services u...,"Under GST, ""intermediary services"" are defined...",Smart Guide,Scope of Intermediary services:\n\n2.1 ‘Interm...,Page 888 to Page 891,GST-Vol. 1,Practice and Procedure Division 4 \nExample 1 ...,1013,NaN,NaN
6,2,Explain the concept of intermediary services u...,"Under GST, ""intermediary services"" are defined...",GST Law and Procedure (Volume 1),Example 1\nA freight forwarder arranges for ex...,Pg-4.209,GST-Vol. 1.,Practice and Procedure Division 4 \nMaintenanc...,1015,NaN,NaN
7,2,Explain the concept of intermediary services u...,"Under GST, ""intermediary services"" are defined...",GST Law and Procedure (Volume 1),DEPARTMENTAL CLARIFICATION\nCircular No. 159/1...,Pg 4.209 to 4.212,"chapter 3, GST-Vol. 1",Place of supply of goods or services or both 4...,1014,NaN,NaN
8,2,Explain the concept of intermediary services u...,"Under GST, ""intermediary services"" are defined...",NaN,NaN,NaN,GST-Vol. 1,he illustrations given in para 4 above are onl...,1015,NaN,NaN
9,2,Explain the concept of intermediary services u...,"Under GST, ""intermediary services"" are defined...",NaN,NaN,NaN,"Chapter3, GST-Vol. 1",lace of supply of goods or services or both 4....,1012,NaN,NaN


In [146]:
df.drop_duplicates(subset=['Chunks (old)', 'Chunks (new)'], inplace=True)

In [155]:

for index, row in df.head(10).iterrows():
    old_chunk = row['Chunks (old)']
    new_chunk = row['Chunks (new)']
    question = row['Question']
    ideal_answer = row['Answer']

    if pd.isna(ideal_answer):
        continue

    if old_chunk:
        old_inputs = [{"role": "system", "content": input_prompt.format(context=old_chunk)},
                {"role": "user", "content": question}]
        old_rag_data.append({
            "input": old_inputs,
            "ideal": ideal_answer
        })
    if new_chunk:
        new_inputs = [{"role": "system", "content": input_prompt.format(context=new_chunk)},
                {"role": "user", "content": question}]
        new_rag_data.append({
            "input": new_inputs,
            "ideal": ideal_answer
        })


In [156]:
old_rag_data

[{'input': [{'role': 'system',
    'content': 'As an expert legal consultant, provide a detailed response to the question below, considering the given context: Section 2(1) of the CGST Act, 2017\n\n“Actionable claim” shall have the same meaning as assigned to it in section 3 of the Transfer of Property Act, 1882\n\nSection 3 of the Transfer of Property Act, 1882\n\n“Actionable claim” means a claim to any debt, other than a debt secured by mortgage of immovable property or by hypothecation or pledge of movable property, or to any beneficial interest in movable property not in the possession, either actual or constructive, of the claimant, which the civil courts recognize as affording grounds for relief, whether such debt or beneficial interest be existent, accruing, conditional or contingent.'},
   {'role': 'user',
    'content': 'What are the GST implications of actionable claims under GST?'}],
  'ideal': 'The taxability of ‘actionable claims’ under GST has changed since the inception 

In [157]:
new_rag_data

[{'input': [{'role': 'system',
    'content': 'As an expert legal consultant, provide a detailed response to the question below, considering the given context: Practice and Proced ure Division 4 \\n(ii) Further, in case of transfer of tenancy rights, a part of the consideration for such transfer accrues \\nto the out going tenant, whether such supplies will also attract GST? \\n2. The issue has been examined. The transfer of tenancy rights against tenancy premium which is \\nalso known as “pagadi system” is prevalent in some States. In this system the tenant acquires, tenancy \\nrights in the property against payment of tenancy premium ( pagadi). The landlord may be owner of the \\nproperty but the possession of the same lies with the tenant. The tenant pays periodic rent to the \\nlandlord as long as he occupies the property. The tenant also usually has the option to sell the tenancy \\nright of t he said property and in such a case has to share a percentage of the proceed with owner 

In [158]:
len(old_rag_data), len(new_rag_data)

(10, 10)

In [159]:
save_dir = "/Users/varadkhonde/Work/evals/evals/registry/data/rag_metrics"
with open(os.path.join(save_dir, 'old_rag_data.jsonl'), 'w') as f:
    for item in old_rag_data:
        f.write(json.dumps(item) + '\n')
with open(os.path.join(save_dir, 'new_rag_data.jsonl'), 'w') as f:
    for item in new_rag_data:
        f.write(json.dumps(item) + '\n')

In [122]:
log_file_path = "/tmp/evallogs/241004110225D6OJHLBU_gpt-3.5-turbo_rag-metrics.jsonl"

In [123]:
# processing the log events generated by oaieval

with open(log_file_path, "r") as f:
    events_df = pd.read_json(f, lines=True)

In [124]:
events_df.head()

,spec,final_report,run_id,event_id,sample_id,type,data,created_by,created_at
0,"{'completion_fns': ['gpt-3.5-turbo'], 'eval_na...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,NaN,"{'counts/Bad': 2, 'counts/Good': 23, 'score': ...",241004110225D6OJHLBU,NaN,NaN,NaN,NaN,NaN,NaT
2,NaN,NaN,241004110225D6OJHLBU,0.0,rag-metrics.dev.8,sampling,"{'prompt': [{'role': 'system', 'content': 'As ...",,2024-10-04 11:02:35.365806+00:00
3,NaN,NaN,241004110225D6OJHLBU,1.0,rag-metrics.dev.33,sampling,"{'prompt': [{'role': 'system', 'content': 'As ...",,2024-10-04 11:02:36.543733+00:00
4,NaN,NaN,241004110225D6OJHLBU,2.0,rag-metrics.dev.25,sampling,"{'prompt': [{'role': 'system', 'content': 'As ...",,2024-10-04 11:02:36.893843+00:00


In [125]:
events_df.dropna(subset=['final_report']).iloc[0]['final_report']

{'counts/Bad': 2, 'counts/Good': 23, 'score': 0.92}

In [126]:
events_df.iloc[2][['run_id', 'event_id', 'sample_id', 'type', 'data', 'created_at']]

run_id                                     241004110225D6OJHLBU
event_id                                                    0.0
sample_id                                     rag-metrics.dev.8
type                                                   sampling
data          {'prompt': [{'role': 'system', 'content': 'As ...
created_at                     2024-10-04 11:02:35.365806+00:00
Name: 2, dtype: object

In [127]:
# Inspect metrics where choice is made and print only the prompt, result, and expected result if the choice is incorrect
for i, row in events_df[events_df['type'] == 'metrics'].iterrows():
    if row['data']['choice'] == 'Bad':
        # Get the previous row's data, which contains the prompt and the expected result
        prev_row = events_df.iloc[i-1]
        
        prompt = prev_row['data']['prompt'][0]['content'] if 'prompt' in prev_row['data'] and len(prev_row['data']['prompt']) > 0 else "Prompt not available"
        expected_result = prev_row['data'].get('ideal', 'Expected result not provided')
        
        # Current row's data will be the actual result
        result = row['data'].get('completion', 'Actual result not provided')
        
        print(prompt)
        print("-" * 40)

You are comparing a submitted AI-generated answer to an expert answer for a given question. Your task is to evaluate the submission based on several metrics. Below is the provided information:

[BEGIN DATA]
************
[Question]: As an expert legal consultant, provide a detailed response to the question below, considering the given context: In Re: Clay Craft (India) Pvt. Ltd 2021 (46) G.S.T.L. 154 (App. A.A.R. -
GST-Raj.), “we find that remuneration, if any, paid by the appellant to the
independent directors or those directors who are not the employee of the
appellant is taxable in hands of the appellant, on reverse charge basis. Further,
the part of Director’s remuneration which are declared as Salaries in the books of
the appellant and subjected to TDS under Section 192 of the IT Act, are not
taxable being consideration for services by an employee to the employer in the
course of or in relation to his employment in terms of Schedule III of the CGST
Act, 2017. The part of employee D